In [1]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

import reverse_geocoder as rg
import pycountry

import warnings
warnings.filterwarnings("ignore")

In [2]:
tce_df = pd.read_csv('./datasets/tce/TCE-DAT_2015-exposure_1950-2015.csv',skiprows=6)
tce_df = tce_df[tce_df['year']>=1980]
tce_df.reset_index(drop=True,inplace=True)
tce_df

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets
0,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,647217.0,2.395292e+09,14151.0,4.985341e+07,0.0,0.000000e+00
1,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,511694.0,5.355011e+10,308434.0,2.668010e+10,0.0,0.000000e+00
2,1980,1980033S12161,UNNAMED,NaN,SP,multi,SLB,25.2,49.0,2918.0,1.350719e+07,0.0,0.000000e+00,0.0,0.000000e+00
3,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,1294326.0,4.797279e+10,650256.0,2.337967e+10,0.0,0.000000e+00
4,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,128330.0,6.126082e+09,0.0,0.000000e+00,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2015,2015309N14067,MEGH,NaN,NI,multi,SOM,39.0,75.8,226629.0,6.596252e+06,40110.0,1.131492e+06,0.0,0.000000e+00
3128,2015,2015309N14067,MEGH,NaN,NI,multi,YEM,61.6,119.7,59993.0,1.188943e+08,57685.0,1.099661e+08,49016.0,9.060994e+07
3129,2015,2015312N11083,UNNAMED,NaN,NI,single,IND,19.4,37.8,846968.0,4.957173e+09,0.0,0.000000e+00,0.0,0.000000e+00
3130,2015,2015344N07145,MELOR,NaN,WP,multi,PHL,65.9,128.1,51335904.0,4.594407e+11,18618400.0,1.808132e+11,4607241.0,4.602029e+10


In [4]:
bego_df = pd.read_csv('./datasets/ibtracs/df_IBTrACS_ISO.csv')
bego_df = bego_df[(bego_df['year']>= 1980)&(bego_df['year']<2016)]
bego_df.reset_index(drop=True,inplace=True)
bego_df.drop(columns=['Unnamed: 0','Unnamed: 0.1'],inplace=True)
bego_df

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,country,ISO
0,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 00:00:00,TS,-16.77,138.47,...,NaN,5,145,1980,1,4,0,0,AU,AUS
1,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 03:00:00,TS,-16.97,138.60,...,NaN,4,147,1980,1,4,3,0,AU,AUS
2,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 06:00:00,TS,-17.13,138.72,...,NaN,4,138,1980,1,4,6,0,AU,AUS
3,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 09:00:00,TS,-17.24,138.85,...,NaN,3,126,1980,1,4,9,0,AU,AUS
4,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 12:00:00,TS,-17.33,139.00,...,NaN,3,127,1980,1,4,12,0,AU,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16460,2015355S16136,2016,110,SP,EA,NOT_NAMED,2015-12-31 09:00:00,NR,-22.73,137.07,...,NaN,7,144,2015,12,31,9,0,AU,AUS
16461,2015355S16136,2016,110,SP,EA,NOT_NAMED,2015-12-31 12:00:00,NR,-23.05,137.32,...,NaN,8,144,2015,12,31,12,0,AU,AUS
16462,2015355S16136,2016,110,SP,EA,NOT_NAMED,2015-12-31 15:00:00,NR,-23.39,137.60,...,NaN,9,144,2015,12,31,15,0,AU,AUS
16463,2015355S16136,2016,110,SP,EA,NOT_NAMED,2015-12-31 18:00:00,NR,-23.75,137.89,...,NaN,9,143,2015,12,31,18,0,AU,AUS


In [5]:
#IBTrACS id's found in the TCE dataset
ibtracs_id = tce_df['IBTrACS_ID'].unique()

#here we store the rows where DIST2LAND in bego_df is minimum
test_list = []

#loop over the IBTrACS id's found in the TCE dataset
for ib in ibtracs_id:
    aux_df = tce_df[tce_df['IBTrACS_ID']==ib]
    #list of countries with the same IBTrACS id
    countries = aux_df['ISO3'].unique()
    #loop over the countries with the same IBTrACS id
    for country in countries:
        aux_df = bego_df[(bego_df['SID']==ib)&(bego_df['ISO']==country)]
        #here we extract the row in bego_df, where DIST2LAND is minimum
        try:
            idxmin = aux_df['DIST2LAND'].idxmin()
            row    = aux_df.loc[idxmin].values
        except:
            #if the record in bego_df is not found, we create an array of NaN
            row = np.empty((167,))
            row[:] = np.nan
        #here we colect the results     
        test_list.append(row)
        
#we convert test_list to an array        
test_array = np.stack(test_list)
test_array.shape

(3132, 167)

In [6]:
prev_df = pd.DataFrame(data=test_array,columns=bego_df.columns)

In [7]:
result_df = pd.concat([tce_df,prev_df],axis=1)
result_df = result_df[result_df['SID'].notna()]

In [8]:
result_df

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,country,ISO
1,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,511694.0,...,NaN,5,145,1980,1,4,0,0,AU,AUS
5,1980,1980069S12161,SINA,NaN,SP,multi,NZL,44.4,86.3,1154353.0,...,NaN,30,139,1980,3,15,0,0,NZ,NZL
6,1980,1980073S09133,DORIS,NaN,SI,single,AUS,28.7,55.7,14390.0,...,NaN,3,264,1980,3,17,12,0,AU,AUS
8,1980,1980076N06148,UNNAMED,NaN,WP,single,PHL,25.8,50.2,4676393.0,...,NaN,11,293,1980,3,23,15,0,PH,PHL
12,1980,1980137N04158,FORREST,NaN,WP,multi,PHL,40.5,78.8,44312971.0,...,NaN,11,303,1980,5,25,3,0,PH,PHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,2015,2015293N13266,PATRICIA,NaN,EP,single,MEX,78.0,151.6,12359952.0,...,NaN,14,20,2015,10,23,23,0,MX,MEX
3126,2015,2015301N11065,CHAPALA,NaN,NI,multi,YEM,46.6,90.6,1467373.0,...,NaN,10,273,2015,11,3,3,0,YE,YEM
3128,2015,2015309N14067,MEGH,NaN,NI,multi,YEM,61.6,119.7,59993.0,...,NaN,4,0,2015,11,10,9,0,YE,YEM
3130,2015,2015344N07145,MELOR,NaN,WP,multi,PHL,65.9,128.1,51335904.0,...,NaN,11,279,2015,12,14,9,0,PH,PHL
